In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#import pyod

from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.metrics import f1_score,classification_report,confusion_matrix,ConfusionMatrixDisplay,recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

#from xgboost import XGBClassifier
#from imblearn.over_sampling import SMOTE
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/content/Bank_Personal_Loan_Modelling(1).csv")
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [ ]:
def IQR(data,treshold=1.5):
    q1 = np.percentile(data,25)
    q3 = np.percentile(data,75)

    iqr = q3 - q1

    lower = q1 - treshold * iqr
    upper = q3 + treshold * iqr

    return lower , upper

In [ ]:
# removing outliers by IQR method

lower_income, upper_income = IQR(df["Income"])

# beacuse we dont have negative values for this columns
# therefor we did not calculate the lower limit
print(f"Upper limit for Income:{upper_income}")

Upper limit for Income:186.5


In [ ]:
# drop ID column
df.drop(["ID","ZIP Code"],axis="columns",inplace=True)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
# we reset index because we droped outliers and index needs to be reset
df.reset_index(drop=True,inplace=True)

In [ ]:
x = df.drop("Personal Loan",axis="columns")
y = df["Personal Loan"]

In [ ]:
import numpy as np

# ---------------- Manual oversampling function ----------------
def oversample(x, y):
    unique, counts = np.unique(y, return_counts=True)
    max_count = max(counts)
    x_res, y_res = [], []

    for cls in unique:
        x_cls = x[y == cls]
        y_cls = y[y == cls]
        reps = max_count // len(x_cls)
        rem = max_count % len(x_cls)
        x_res.append(np.tile(x_cls, (reps, 1)))
        y_res.append(np.tile(y_cls, reps))
        if rem > 0:
            x_res.append(x_cls[:rem])
            y_res.append(y_cls[:rem])

    x_res = np.vstack(x_res)
    y_res = np.hstack(y_res)

    # Shuffle
    idx = np.arange(len(x_res))
    np.random.shuffle(idx)
    return x_res[idx], y_res[idx]

# ---------------- Apply oversampling ----------------
# x and y contain your original dataset
x_bal, y_bal = oversample(x, y)

# ---------------- Separate for model training ----------------
x = x_bal
y = y_bal

# Optional: check new class counts
unique, counts = np.unique(y, return_counts=True)
print("Balanced class counts:", dict(zip(unique, counts)))


Balanced class counts: {np.int64(0): np.int64(4520), np.int64(1): np.int64(4520)}


In [ ]:
import pandas as pd

y_series = pd.Series(y)
print(y_series.value_counts())

1    4520
0    4520
Name: count, dtype: int64


In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y_series,train_size=0.8,random_state=42)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.metrics import classification_report, confusion_matrix

# ----------------------------- 1. Client Split -----------------------------
def split_clients(X, y, n_clients=4):
    idx = np.arange(len(X))
    np.random.shuffle(idx)
    return [(X[s], y[s]) for s in np.array_split(idx, n_clients)]

# ----------------------------- 2. LSTM Model -----------------------------
class SmallLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, n_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, n_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.relu(self.fc1(out))
        return self.fc2(out)

# ----------------------------- 3. FedAvg -----------------------------
def fedavg(global_model, local_models, sizes):
    total = sum(sizes)
    global_dict = global_model.state_dict()

    for key in global_dict:
        global_dict[key] = sum(
            local_models[i].state_dict()[key] * sizes[i]
            for i in range(len(local_models))
        ) / total

    global_model.load_state_dict(global_dict)
    return global_model

# ----------------------------- 4. FL Training -----------------------------
def run_federated_learning(
    xtrain, ytrain, xtest, ytest,
    device_name,
    cpu_threads=None,
    rounds=40,
    epochs=2,
    batch_size=16
):
    print("\n" + "="*70)
    print(f"RUN MODE: {device_name}")
    print("="*70)

    # -------- Device & CPU threading --------
    if cpu_threads is not None:
        torch.set_num_threads(cpu_threads)
        print(f"CPU Threads Used: {torch.get_num_threads()}")

    device = torch.device(device_name)
    print("Device:", device)

    # -------- Data preparation --------
    xtrain = np.array(xtrain)
    xtest  = np.array(xtest)
    ytrain = np.array(ytrain)
    ytest  = np.array(ytest)

    if xtrain.ndim == 2:
        xtrain = xtrain[:, :, None]
        xtest  = xtest[:, :, None]

    xtrain = torch.tensor(xtrain, dtype=torch.float32)
    ytrain = torch.tensor(ytrain, dtype=torch.long)
    xtest  = torch.tensor(xtest, dtype=torch.float32).to(device)
    ytest  = torch.tensor(ytest, dtype=torch.long).to(device)

    clients = split_clients(xtrain, ytrain)
    n_classes = len(np.unique(ytrain.numpy()))

    global_model = SmallLSTM(1, 32, n_classes).to(device)
    criterion = nn.CrossEntropyLoss()

    start_total = time.time()

    # -------- Federated Rounds --------
    for r in range(rounds):
        local_models, sizes = [], []
        round_start = time.time()

        for Xc, yc in clients:
            Xc = Xc.to(device)
            yc = yc.to(device)

            lm = SmallLSTM(1, 32, n_classes).to(device)
            lm.load_state_dict(global_model.state_dict())
            optimizer = optim.Adam(lm.parameters(), lr=0.001)

            lm.train()
            for _ in range(epochs):
                for i in range(0, len(Xc), batch_size):
                    xb = Xc[i:i+batch_size]
                    yb = yc[i:i+batch_size]
                    optimizer.zero_grad()
                    loss = criterion(lm(xb), yb)
                    loss.backward()
                    optimizer.step()

            local_models.append(lm)
            sizes.append(len(Xc))

        global_model = fedavg(global_model, local_models, sizes)

        global_model.eval()
        with torch.no_grad():
            preds = global_model(xtest).argmax(1)
            acc = (preds == ytest).float().mean().item()

        print(f"Round {r+1:02d} | Accuracy: {acc:.4f} | Time: {time.time()-round_start:.2f}s")

    total_time = time.time() - start_total
    print(f"\nTOTAL TRAINING TIME ({device_name}): {total_time:.2f} seconds")

    print("\nClassification Report:\n",
          classification_report(ytest.cpu(), preds.cpu()))
    print("Confusion Matrix:\n",
          confusion_matrix(ytest.cpu(), preds.cpu()))

    return total_time


In [ ]:
serial_time = run_federated_learning(
    xtrain, ytrain, xtest, ytest,
    device_name="cpu",
    cpu_threads=1,      # ← SERIAL CPU
    rounds=40,
    epochs=2
)



RUN MODE: cpu
CPU Threads Used: 1
Device: cpu
Round 01 | Accuracy: 0.8368 | Time: 3.53s
Round 02 | Accuracy: 0.8971 | Time: 4.47s
Round 03 | Accuracy: 0.8949 | Time: 2.38s
Round 04 | Accuracy: 0.9010 | Time: 2.85s
Round 05 | Accuracy: 0.9054 | Time: 2.26s
Round 06 | Accuracy: 0.9093 | Time: 2.18s
Round 07 | Accuracy: 0.9126 | Time: 2.08s
Round 08 | Accuracy: 0.9159 | Time: 1.83s
Round 09 | Accuracy: 0.9220 | Time: 2.68s
Round 10 | Accuracy: 0.9253 | Time: 3.29s
Round 11 | Accuracy: 0.9281 | Time: 2.07s
Round 12 | Accuracy: 0.9292 | Time: 2.16s
Round 13 | Accuracy: 0.9298 | Time: 1.79s
Round 14 | Accuracy: 0.9298 | Time: 1.81s
Round 15 | Accuracy: 0.9336 | Time: 1.75s
Round 16 | Accuracy: 0.9403 | Time: 1.72s
Round 17 | Accuracy: 0.9441 | Time: 1.74s
Round 18 | Accuracy: 0.9458 | Time: 2.21s
Round 19 | Accuracy: 0.9463 | Time: 2.01s
Round 20 | Accuracy: 0.9491 | Time: 1.78s
Round 21 | Accuracy: 0.9475 | Time: 1.77s
Round 22 | Accuracy: 0.9541 | Time: 1.75s
Round 23 | Accuracy: 0.9580 |

In [ ]:
multi_cpu_time = run_federated_learning(
    xtrain, ytrain, xtest, ytest,
    device_name="cpu",
    cpu_threads=8,      # ← MULTI-CORE CPU
    rounds=40,
    epochs=2
)



RUN MODE: cpu
CPU Threads Used: 8
Device: cpu
Round 01 | Accuracy: 0.8861 | Time: 5.70s
Round 02 | Accuracy: 0.8921 | Time: 6.61s
Round 03 | Accuracy: 0.8955 | Time: 5.65s
Round 04 | Accuracy: 0.8988 | Time: 6.57s
Round 05 | Accuracy: 0.9027 | Time: 5.63s
Round 06 | Accuracy: 0.9049 | Time: 6.24s
Round 07 | Accuracy: 0.9137 | Time: 5.57s
Round 08 | Accuracy: 0.9209 | Time: 6.37s
Round 09 | Accuracy: 0.9215 | Time: 6.18s
Round 10 | Accuracy: 0.9264 | Time: 6.44s
Round 11 | Accuracy: 0.9253 | Time: 5.47s
Round 12 | Accuracy: 0.9220 | Time: 6.51s
Round 13 | Accuracy: 0.9248 | Time: 6.45s
Round 14 | Accuracy: 0.9314 | Time: 6.41s
Round 15 | Accuracy: 0.9292 | Time: 5.56s
Round 16 | Accuracy: 0.9336 | Time: 6.26s
Round 17 | Accuracy: 0.9381 | Time: 5.50s
Round 18 | Accuracy: 0.9364 | Time: 6.35s
Round 19 | Accuracy: 0.9381 | Time: 5.63s
Round 20 | Accuracy: 0.9397 | Time: 6.29s
Round 21 | Accuracy: 0.9430 | Time: 5.55s
Round 22 | Accuracy: 0.9458 | Time: 6.31s
Round 23 | Accuracy: 0.9463 |

In [ ]:
if torch.cuda.is_available():
    gpu_time = run_federated_learning(
        xtrain, ytrain, xtest, ytest,
        device_name="cuda",   # ← GPU PARALLELISM
        cpu_threads=None,
        rounds=40,
        epochs=2
    )
else:
    print("CUDA not available")



RUN MODE: cuda
Device: cuda
Round 01 | Accuracy: 0.8850 | Time: 1.84s
Round 02 | Accuracy: 0.8955 | Time: 1.77s
Round 03 | Accuracy: 0.8999 | Time: 1.80s
Round 04 | Accuracy: 0.9004 | Time: 1.77s
Round 05 | Accuracy: 0.9065 | Time: 1.94s
Round 06 | Accuracy: 0.9087 | Time: 2.14s
Round 07 | Accuracy: 0.9121 | Time: 1.78s
Round 08 | Accuracy: 0.9198 | Time: 1.76s
Round 09 | Accuracy: 0.9209 | Time: 1.79s
Round 10 | Accuracy: 0.9248 | Time: 1.75s
Round 11 | Accuracy: 0.9303 | Time: 1.78s
Round 12 | Accuracy: 0.9375 | Time: 2.21s
Round 13 | Accuracy: 0.9403 | Time: 1.91s
Round 14 | Accuracy: 0.9408 | Time: 1.80s
Round 15 | Accuracy: 0.9458 | Time: 1.79s
Round 16 | Accuracy: 0.9513 | Time: 1.76s
Round 17 | Accuracy: 0.9569 | Time: 1.75s
Round 18 | Accuracy: 0.9585 | Time: 1.97s
Round 19 | Accuracy: 0.9591 | Time: 2.09s
Round 20 | Accuracy: 0.9613 | Time: 1.76s
Round 21 | Accuracy: 0.9652 | Time: 1.75s
Round 22 | Accuracy: 0.9685 | Time: 1.78s
Round 23 | Accuracy: 0.9657 | Time: 1.75s
Round